In [1]:
!pip install torch torchvision torchaudio pytorch-lightning scikit-learn pandas soundfile tensorboard tensorboardX

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.0/823.0 kB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torchvision.models as models
from sklearn.metrics import confusion_matrix  # New import for confusion matrix
from pytorch_lightning.loggers import TensorBoardLogger
import tensorboard
import tensorboardX
from pytorch_lightning.loggers import CSVLogger
logger = CSVLogger("logs", name="my_model")

In [4]:
# Set torchaudio backend (this call is deprecated but needed for .au files)
torchaudio.set_audio_backend("soundfile")

<ipython-input-4-a7bb97d14381>:2: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


In [5]:
# ----------------------------
# Step 1: Check GPU availability
# ----------------------------
print("Is GPU available?", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))

Is GPU available? True
GPU name: Tesla T4


In [10]:
# ----------------------------
# Step 2: Load dataset from Google Drive
# ----------------------------
# Adjust the path below if your folder structure is different.
dataset_path = '../datasets/GTZAN/'
file_paths = []
genres = []
for root, _, files in os.walk(dataset_path):
    for file in files:
        if file.endswith('.au'):
            file_paths.append(os.path.join(root, file))
            genres.append(os.path.basename(root))  # Genre inferred from folder name

df = pd.DataFrame({'file_path': file_paths, 'genre': genres})
print("Dataset loaded. Total files:", len(df))

Dataset loaded. Total files: 1000


In [11]:
# ----------------------------
# Step 3: Label encoding for genres
# ----------------------------
global_le = LabelEncoder()
global_le.fit(df['genre'])
num_classes = len(global_le.classes_)
print("Number of classes:", num_classes)

Number of classes: 10


In [12]:
# ----------------------------
# Step 4: Define the dataset class to compute mel-spectrograms
# ----------------------------
class GTZANDataset(Dataset):
    def __init__(self, df, label_encoder, duration=10, n_mels=128, hop_length=256):
        self.df = df.reset_index(drop=True)
        self.duration = duration            # seconds of audio to load
        self.n_mels = n_mels                # number of mel bins
        self.hop_length = hop_length        # hop length for mel-spectrogram
        self.sample_rate = 22050            # standard sample rate
        self.le = label_encoder             # shared label encoder

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        file_path = row['file_path']
        try:
            waveform, sr = torchaudio.load(file_path)
            # Convert to mono if stereo
            if waveform.shape[0] > 1:
                waveform = torch.mean(waveform, dim=0, keepdim=True)
            target_samples = self.sample_rate * self.duration
            if waveform.shape[1] > target_samples:
                waveform = waveform[:, :target_samples]
            else:
                waveform = F.pad(waveform, (0, target_samples - waveform.shape[1]))
            # Compute mel-spectrogram and convert to decibel scale
            mel_transform = torchaudio.transforms.MelSpectrogram(
                sample_rate=self.sample_rate,
                n_mels=self.n_mels,
                n_fft=2048,
                hop_length=self.hop_length
            )
            mel_spec = mel_transform(waveform)
            mel_spec = torchaudio.transforms.AmplitudeToDB()(mel_spec)
            # Rearrange to shape (time_steps, n_mels)
            mel_spec = mel_spec.squeeze(0).T
            label = torch.tensor(self.le.transform([row['genre']])[0], dtype=torch.long)
            return mel_spec, label
        except Exception as e:
            print(f"Error loading {file_path}: {e}")
            default_mel_spec = torch.zeros((self.duration * self.sample_rate // self.hop_length + 1, self.n_mels))
            return default_mel_spec, torch.tensor(0, dtype=torch.long)


In [13]:
# ----------------------------
# Step 5: Split the data into training and validation sets
# ----------------------------
df_train, df_val = train_test_split(df, test_size=0.2, random_state=42, stratify=df['genre'])
train_dataset = GTZANDataset(df_train, label_encoder=global_le, duration=10, n_mels=128, hop_length=256)
val_dataset = GTZANDataset(df_val, label_encoder=global_le, duration=10, n_mels=128, hop_length=256)

In [14]:
# ----------------------------
# Step 6: Create DataLoaders
# ----------------------------
batch_size = 8  # Adjust based on your GPU memory
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)


In [15]:
# ----------------------------
# Step 7: Define the Transfer Learning Model using Pretrained EfficientNet-B3
# ----------------------------
class MusicEfficientNetB7(pl.LightningModule):
    def __init__(self, num_classes, learning_rate=1e-3):
        super(MusicEfficientNetB7, self).__init__()
        self.save_hyperparameters()
        self.learning_rate = learning_rate
        # Load pretrained EfficientNet-B7
        self.model = models.efficientnet_b7(pretrained=True)
        # Modify the first conv layer to accept 1 channel instead of 3
        self.model.features[0][0] = nn.Conv2d(
            1,
            self.model.features[0][0].out_channels,
            kernel_size=self.model.features[0][0].kernel_size,
            stride=self.model.features[0][0].stride,
            padding=self.model.features[0][0].padding,
            bias=False
        )
        # Replace the classifier to output the correct number of classes
        in_features = self.model.classifier[1].in_features
        self.model.classifier[1] = nn.Linear(in_features, num_classes)

    def forward(self, x):
        # x shape: (batch, time_steps, n_mels)
        x = x.unsqueeze(1)  # Add channel dimension: (batch, 1, time_steps, n_mels)
        # Resize to 300x300 to match EfficientNet-B3 input requirements
        x = F.interpolate(x, size=(300, 300), mode="bilinear", align_corners=False)
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True)
        return loss

    def on_validation_epoch_start(self):
        # Initialize lists to accumulate predictions and targets
        self.all_preds = []
        self.all_targets = []

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = (preds == y).float().mean()
        # Accumulate predictions and targets
        self.all_preds.append(preds)
        self.all_targets.append(y)
        self.log("val_loss", loss, on_epoch=True, prog_bar=True)
        self.log("val_acc", acc, on_epoch=True, prog_bar=True)
        return loss

    def on_validation_epoch_end(self):
        # Concatenate all predictions and targets
        all_preds = torch.cat(self.all_preds)
        all_targets = torch.cat(self.all_targets)
        cm = confusion_matrix(all_targets.cpu().numpy(), all_preds.cpu().numpy())
        print("Confusion Matrix:\n", cm)
        # Log the confusion matrix if the logger supports add_text
        if self.logger is not None and hasattr(self.logger.experiment, "add_text"):
            self.logger.experiment.add_text("Confusion Matrix", str(cm), self.current_epoch)
        else:
            print("Logger does not support add_text. Confusion Matrix not logged.")

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=3)
        return {"optimizer": optimizer, "lr_scheduler": {"scheduler": scheduler, "monitor": "val_loss"}}


In [16]:
# ----------------------------
# Step 8: Initialize the model
# ----------------------------
model = MusicEfficientNetB7(num_classes=num_classes, learning_rate=1e-3)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B7_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B7_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b7_lukemelas-c5b4e57e.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b7_lukemelas-c5b4e57e.pth
100%|██████████| 255M/255M [00:02<00:00, 132MB/s]


In [17]:
# ----------------------------
# Step 9: Configure the Trainer with the CSVLogger
# ----------------------------
trainer = Trainer(
    max_epochs=30,  # More epochs for thorough fine-tuning
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    devices=1,
    precision="16-mixed",  # Mixed precision for faster training on GPU
    accumulate_grad_batches=4,
    log_every_n_steps=10,
    logger=logger,
    num_sanity_val_steps=0
)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [18]:
# ----------------------------
# Step 10: Train and validate the model
# ----------------------------
trainer.fit(model, train_loader, val_loader)
results = trainer.validate(model, dataloaders=val_loader)
print("Validation results:", results)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type         | Params | Mode 
-----------------------------------------------
0 | model | EfficientNet | 63.8 M | train
-----------------------------------------------
63.8 M    Trainable params
0         Non-trainable params
63.8 M    Total params
255.246   Total estimated model params size (MB)
1105      Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Confusion Matrix:
 [[ 8  0  0  0  7  3  1  1  0  0]
 [ 1 16  0  0  0  3  0  0  0  0]
 [ 2  0  6  0  5  0  1  0  5  1]
 [ 0  0  0  0 10  0  0  9  1  0]
 [ 0  0  0  0 15  0  0  5  0  0]
 [ 0  0  0  0  1 18  0  0  1  0]
 [ 0  0  0  0  7  0 13  0  0  0]
 [ 0  0  1  0  0  1  0 18  0  0]
 [ 0  0  0  0  6  0  0  8  6  0]
 [ 1  1  0  0  5  1  6  4  2  0]]
Logger does not support add_text. Confusion Matrix not logged.


Validation: |          | 0/? [00:00<?, ?it/s]

...

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=30` reached.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: |          | 0/? [00:00<?, ?it/s]

Confusion Matrix:
 [[18  0  1  0  0  0  1  0  0  0]
 [ 0 20  0  0  0  0  0  0  0  0]
 [ 0  0 18  0  0  0  0  0  0  2]
 [ 0  0  0 12  3  0  0  2  0  3]
 [ 0  0  0  0 19  0  1  0  0  0]
 [ 0  1  0  0  0 19  0  0  0  0]
 [ 1  0  0  0  0  0 18  0  0  1]
 [ 0  0  2  1  0  0  0 16  1  0]
 [ 0  0  1  2  1  0  0  1 15  0]
 [ 0  0  0  0  0  1  1  0  0 18]]
Logger does not support add_text. Confusion Matrix not logged.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          val_acc          │    0.8650000095367432     │
│         val_loss          │     0.61114501953125      │
└───────────────────────────┴───────────────────────────┘

Validation results: [{'val_loss': 0.61114501953125, 'val_acc': 0.8650000095367432}]
